In [17]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import StateGraph, START, END
from typing import Annotated, TypedDict
import pprint


# --- Define the shared state ---
class GraphState(TypedDict):
    question: str
    context: str
    responses: Annotated[dict, lambda x, y: {**x, **y}]  # merge rule


# --- Define model node ---
def chat_response(model_name: str):
    def _run(state: GraphState):
        model = ChatOllama(model=model_name)
        prompt = ChatPromptTemplate.from_messages([
            (
                "system",
                "You are a thoughtful and supportive assistant who provides emotionally intelligent and practical advice."
            ),
            (
                "user",
                (
                    "Given the context below, provide a calm, healthy, and actionable response.\n\n"
                    "Context: {context}\n\n"
                    "Question: {question}\n\n"
                    "Your response should focus on emotional well-being, mindset improvement, and practical steps."
                ),
            ),
        ])
        chain = prompt | model | StrOutputParser()

        answer = chain.invoke({
            "question": state["question"],
            "context": state["context"],
        })

        return {"responses": {model_name: answer}}

    return _run


# --- Build the graph ---
workflow = StateGraph(GraphState)
workflow.add_node("llama3.1", chat_response("llama3.1"))
workflow.add_node("deepseek-r1", chat_response("deepseek-r1"))

# Run both in parallel
workflow.add_edge(START, "llama3.1")
workflow.add_edge(START, "deepseek-r1")
workflow.add_edge("llama3.1", END)
workflow.add_edge("deepseek-r1", END)

# Compile
graph = workflow.compile()


# --- Run example ---
inputs: GraphState = {
    "question": "What is 1+1?",
    "context": "",
    "responses": {},
}

# Run graph — this returns the **final merged output**
final_state = graph.invoke(inputs)

# Pretty print cleanly
pprint.pprint(final_state)


{'context': '',
 'question': 'What is 1+1?',
 'responses': {'deepseek-r1': "Okay, let's approach this thoughtfully.\n"
                              '\n'
                              'The simple mathematical answer to "What is '
                              '1+1?" is 2. That\'s a basic fact.\n'
                              '\n'
                              "However, I understand you're asking for a "
                              'response focused on emotional well-being and '
                              "mindset, even for such a simple question. Let's "
                              'explore that perspective:\n'
                              '\n'
                              "1.  **Acknowledge the Question:** It's "
                              'perfectly normal to sometimes ask simple '
                              'questions or feel uncertain, even about basic '
                              "things. Don't judge yourself for this. "
                              'Accepting 

In [2]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.language_models import BaseChatModel

from graphstate import GraphState
from workflow import create_graph
from agents.agent import Agent
from agents.hallucination_classifier import HallucinationClassifier
import pprint



# # --- Define model node ---
# def chat_response(model: BaseChatModel):
#     def _run(state: GraphState):
        
#         prompt = ChatPromptTemplate.from_messages([
#             (
#                 "system",
#                 "You are a thoughtful and supportive assistant who provides emotionally intelligent and practical advice."
#             ),
#             (
#                 "user",
#                 (
#                     "Given the context below, provide a calm, healthy, and actionable response.\\n\\n",
#                     "Context: {context}\\n\\n",
#                     "Question: {question}\\n\\n",
#                     "Your response should focus on emotional well-being, mindset improvement, and practical steps."
#                 ),
#             ),
#         ])
#         chain = prompt | model | StrOutputParser()

#         answer = chain.invoke({
#             "question": state["question"],
#             "context": state["context"],
#         })

#         return {"responses": {model_name: answer}}

#     return _run

# Wrap ChatOllama instances with your BaseModel so they expose `execute`
base_models = [
    Agent(ChatOllama(model="llama3.1"), rank=1),
    Agent(ChatOllama(model="llama3.1"), rank=2),
]

hallu_classifier = HallucinationClassifier(ChatOllama(model="llama3.1"))


# --- Run example ---
inputs: GraphState = {
    "question": "What is 1+1?",
    "context": "",
    "responses": {},
}

graph = create_graph(base_models, 
                     hallu_classifier
                     )

# Run graph — this returns the **final merged output**
final_state = graph.invoke(inputs)

# Pretty print cleanly
pprint.pprint(final_state)

{'context': '',
 'question': '',
 'responses': {'BaseModel_1': {'hallu_type': (None, 'UNKNOWN'),
                               'reason': 'This interaction does not contain '
                                         'any hallucination, as the LLM '
                                         'generation only responds to a prompt '
                                         'with a generic statement and awaits '
                                         'further input.',
                               'response': "I'm ready to assist with general "
                                           'question-answering tasks. What is '
                                           'your question?'},
               'BaseModel_2': {'hallu_type': (None, 'UNKNOWN'),
                               'reason': 'This is a normal conversation '
                                         'starter, and there is no evidence of '
                                         'hallucination in the provided '
                 